In [4]:
# import final_dataset.csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor

In [5]:
# Load the dataset
df_features = pd.read_csv('final_dataset.csv')

# Load images into the dataframe


KeyError: 'ID'

In [2]:
# Create a diffusion model
# Split the data into features and target
df_images = df_features['Image']
df_features = df_features.drop('Image', axis=1)

X = df_images
y = df_features

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Create a pipeline
model = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100))

# Fit the model
model.fit(X_train, y_train)

# Predict the target
y_pred = model.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


NameError: name 'df_features' is not defined